# Loading Unified Model output

For convenience, `aeolus` provides a way of keeping loaded and processed data within one object along with extra metadata.
The object is called `Run` (as in "simulation run"). 
The code below provides an example of basic usage of `Run`.

In [1]:
from pathlib import Path

import iris

from aeolus.core import Run

Note that you can use either a single filename or a list of filenames, each of which is either a `str` or (recommended) `pathlib.Path` object.

In [2]:
sample_file = Path.cwd() / "sample_data" / "sample_t1e_2d_mean.pp"

While instantiating `Run`, it is possible to add a short name, a long description of the experiment; and to specify a planet configuration with relevant constants (see "Physical constants" example for more info).

In [3]:
my_run = Run(
    files=sample_file,
    name="t1e_example",
    description="This is some sample data from a UM simulation of tidally-locked Trappist-1e planet.",
    planet="trap1e"  # this reads constants from a JSON file
)

In [4]:
my_run

Constants that have been used in the model:

In [5]:
my_run.const

Trap1eConstants(earth_day [s], stefan_boltzmann [W m-2 K-4], molar_gas_constant [J K-1 mol-1], water_heat_vaporization [m2 s-2], water_molecular_weight [kg mol-1], gravity [m s-2], radius [m], day [s], solar_constant [W m-2], semi_major_axis [au], eccentricity [1], obliquity [degree], dry_air_spec_heat_press [m2 s-2 K-1], dry_air_molecular_weight [kg mol-1])

The loaded data are stored as a `CubeList` under `raw` attribute.

In [6]:
my_run.raw

[<iris 'Cube' of convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of low_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>]

## Minimal processing of loaded data

The next step might involve some clean-up and post-processing of raw data.

This is done by creating a function that takes `iris.cube.CubeList` as its 1st argument and returns another `iris.cube.CubeList` as output.
The function is then passed to `Run.proc_data()` and its output stored as `Run.proc` attribute.

For example, the function `roll_cube_pm180()` imported below takes a `Cube` and rolls its longitudes from 0...360 to -180...180.

In [7]:
from aeolus.grid import roll_cube_pm180, ensure_bounds

It can then be applied to all `raw` cubes.

In [8]:
def _prepare_cubes(cubelist):
    """Post-process data for easier analysis."""
    # Roll cubes
    r_cubes = iris.cube.CubeList()
    for cube in cubelist:
        r_c = roll_cube_pm180(cube)
        ensure_bounds(r_c)  # also, ensure that longitudes and latitudes have bounds
        r_cubes.append(r_c)

    return r_cubes

In [9]:
my_run.proc_data(_prepare_cubes)

In [10]:
my_run.proc

[<iris 'Cube' of convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of low_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>]

Check that it did what expected.

In [11]:
print(my_run.raw.extract_strict("convective_rainfall_flux").coord("longitude"))

DimCoord(array([  1.25,   3.75,   6.25,   8.75,  11.25,  13.75,  16.25,  18.75,
        21.25,  23.75,  26.25,  28.75,  31.25,  33.75,  36.25,  38.75,
        41.25,  43.75,  46.25,  48.75,  51.25,  53.75,  56.25,  58.75,
        61.25,  63.75,  66.25,  68.75,  71.25,  73.75,  76.25,  78.75,
        81.25,  83.75,  86.25,  88.75,  91.25,  93.75,  96.25,  98.75,
       101.25, 103.75, 106.25, 108.75, 111.25, 113.75, 116.25, 118.75,
       121.25, 123.75, 126.25, 128.75, 131.25, 133.75, 136.25, 138.75,
       141.25, 143.75, 146.25, 148.75, 151.25, 153.75, 156.25, 158.75,
       161.25, 163.75, 166.25, 168.75, 171.25, 173.75, 176.25, 178.75,
       181.25, 183.75, 186.25, 188.75, 191.25, 193.75, 196.25, 198.75,
       201.25, 203.75, 206.25, 208.75, 211.25, 213.75, 216.25, 218.75,
       221.25, 223.75, 226.25, 228.75, 231.25, 233.75, 236.25, 238.75,
       241.25, 243.75, 246.25, 248.75, 251.25, 253.75, 256.25, 258.75,
       261.25, 263.75, 266.25, 268.75, 271.25, 273.75, 276.25, 278.7

In [12]:
print(my_run.proc.extract_strict("convective_rainfall_flux").coord("longitude"))

DimCoord(array([-178.75, -176.25, -173.75, -171.25, -168.75, -166.25, -163.75,
       -161.25, -158.75, -156.25, -153.75, -151.25, -148.75, -146.25,
       -143.75, -141.25, -138.75, -136.25, -133.75, -131.25, -128.75,
       -126.25, -123.75, -121.25, -118.75, -116.25, -113.75, -111.25,
       -108.75, -106.25, -103.75, -101.25,  -98.75,  -96.25,  -93.75,
        -91.25,  -88.75,  -86.25,  -83.75,  -81.25,  -78.75,  -76.25,
        -73.75,  -71.25,  -68.75,  -66.25,  -63.75,  -61.25,  -58.75,
        -56.25,  -53.75,  -51.25,  -48.75,  -46.25,  -43.75,  -41.25,
        -38.75,  -36.25,  -33.75,  -31.25,  -28.75,  -26.25,  -23.75,
        -21.25,  -18.75,  -16.25,  -13.75,  -11.25,   -8.75,   -6.25,
         -3.75,   -1.25,    1.25,    3.75,    6.25,    8.75,   11.25,
         13.75,   16.25,   18.75,   21.25,   23.75,   26.25,   28.75,
         31.25,   33.75,   36.25,   38.75,   41.25,   43.75,   46.25,
         48.75,   51.25,   53.75,   56.25,   58.75,   61.25,   63.75,
         66

Note that the longitude coordinate is not only shifted by 180 degrees, but has automatically calculated `bounds`.

In addition, note that `coord_system` in `proc` cubes is different, because it used `Run.const` attribute to redefine the planet radius correctly.
By default the loaded raw data has Earth radius in its `coord_system`, so certain calculations (e.g. grid cell areas) might be incorrect.